## Deploy  <font color='red'>PACE-ML: Feature Selection</font> Model Package from AWS Marketplace 


### <font color='red'> The solution runs user - specified feature selection tasks on the user data and provides relevant features as output.</font>

<font>PACE-ML Feature Selection will run machine learning related feature selection operations on the user provided data. The feature selection tasks to execute on the data can be provided by user in a separate config file. This will simplify the task of feature selection for a data scientist where in the user will only have to specify select few parameter to generate the correct output instead of writing specific code for each individual feature selection tasks.</font>

This sample notebook shows you how to deploy <font color='red'>[Explainable AI: Algorithm Bias Detection](http:\\)</font> using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to <font color='red'>[Explainable AI: Algorithm Bias Detection](http:\\)</font>. If so, skip step: [Subscribe to the model package](http:\\)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> [Explainable AI: Algorithm Bias Detection](http:\\).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='your-arn-number'

<font color='red'> For Seller to update: Add all necessary imports in following cell, 
If you need specific packages to be installed, # try to provide them in this section, in a separate cell. </font>

In [2]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
#from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
#import urllib.request
import numpy as np

No handlers could be found for logger "sagemaker"


In [3]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

<font color='red'>For Seller to update: update values for four variables in following cell. 
Specify a model/endpoint name using only alphanumeric characters. </font>

In [4]:
model_name='paceml-feature-selection'

content_type='application/zip'

real_time_inference_instance_type='ml.m5.xlarge'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [5]:

def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type)

#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

#Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

<font color='red'>For Seller to update: Add code snippet here that reads the input from 'data/input/real-time/' directory 
and converts it into format expected by the endpoint.</font>

In [11]:
from zipfile import ZipFile
import pandas as pd
file_name = './data/input/real-time/sample_data.zip'
def read_zip_file(zipfile_obj):
    try:
        file_names = zipfile_obj.namelist()
        csv_file_name = [f for f in file_names if 'csv' in f][0]
        json_file_name = [f for f in file_names if 'json' in f][0]
        json_file_content = json.load(zipfile_obj.open(json_file_name))
        data = pd.read_csv(zipfile_obj.open(csv_file_name))
        return data, json_file_content
    except Exception as e:
        print('There was some error:', str(e))
    return None, None

zipfile_obj = ZipFile(file_name)
data_df, config_dict = read_zip_file(zipfile_obj)

In [12]:
print(data_df.head(10))

   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   
5        12.45         15.70           82.57      477.1          0.12780   
6        18.25         19.98          119.60     1040.0          0.09463   
7        13.71         20.83           90.20      577.9          0.11890   
8        13.00         21.82           87.50      519.8          0.12730   
9        12.46         24.04           83.97      475.9          0.11860   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760         0.30010              0.14710         0.2419   
1           0

In [16]:
config_dict

{u'feat_selection': {u'apply_feature_selection': False,
  u'apply_pca': False,
  u'feature_selection_method': u'classic',
  u'feature_selections_top_features_to_keep': 0.8,
  u'maximum_correlation': 0.9,
  u'pca_method': u'liner',
  u'pca_variance_retained_or_number_of_components': 0.99,
  u'remove_multicollinearity': False},
 u'target_variable': u'labels'}

<Add code snippet that shows the payload contents>

<font color='red'>For Seller to update: Ensure that file_name variable points to the payload you created. 
Ensure that output_file_name variable points to a file-name in which output of real-time inference needs to be stored.</font>

In [20]:
output_file_name = 'output.txt'

#### C. Perform real-time inference

<font color='red'>For Seller to update: review/update file_name, output_file name, custom attributes in following AWS CLI to perform a real-time inference using the payload file you created from 2.B </font>

In [8]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name 'paceml-feature-selection' \
    --body fileb://data/input/real-time/sample_data.zip \
    --content-type 'application/zip' \
    --region us-east-2 \
    output.csv

{
    "InvokedProductionVariant": "AllTraffic", 
    "ContentType": "text/csv; charset=utf-8"
}


#### D. Visualize output

In [9]:
import pandas as pd
output = pd.read_csv("./data/output/real-time/output.csv")
print(output.head(10))

   Unnamed: 0  labels  mean radius  mean texture  mean perimeter  mean area  \
0           0     0.0        17.99         10.38          122.80     1001.0   
1           1     0.0        20.57         17.77          132.90     1326.0   
2           2     0.0        19.69         21.25          130.00     1203.0   
3           3     0.0        11.42         20.38           77.58      386.1   
4           4     0.0        20.29         14.34          135.10     1297.0   
5           5     0.0        12.45         15.70           82.57      477.1   
6           6     0.0        18.25         19.98          119.60     1040.0   
7           7     0.0        13.71         20.83           90.20      577.9   
8           8     0.0        13.00         21.82           87.50      519.8   
9           9     0.0        12.46         24.04           83.97      475.9   

   mean smoothness  mean compactness  mean concavity  mean concave points  \
0          0.11840           0.27760         0.30010 

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [21]:
predictor=sage.RealTimePredictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [17]:
#upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/paceml-feature-selection


In [18]:
#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

........................
Starting the inference server with 2 workers.
2021/01/19 10:29:09 [crit] 13#13: *1 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [19/Jan/2021:10:29:09 +0000] "GET /ping HTTP/1.1" 502 182 "-" "Go-http-client/1.1"
2021/01/19 10:29:09 [crit] 13#13: *3 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [19/Jan/2021:10:29:09 +0000] "GET /ping HTTP/1.1" 502 182 "-" "Go-http-client/1.1"
[2021-01-19 10:29:09 +0000] [12] [INFO] Starting gunicorn 20.0.4
[2021-01-19 10:29:09 +0000] [12] [INFO] Listening at: unix:/tmp/gunicorn.sock (12)
[

In [19]:
#output is available on following path
transformer.output_path

's3://sagemaker-us-east-2-786796469737/paceml-feature-selection-2021-01-18-17--2021-01-19-10-25-21-443'

In [21]:
s3_conn = boto3.client("s3")
bucket_name="your-bucket-name"
with open('./data/output/batch/output.csv', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, 'folder-name'+'/sample_data.zip.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [22]:
output_df = pd.read_csv("./data/output/batch/output.csv")
#print(output.read())
print(output_df.head(10))

   Unnamed: 0  labels  mean radius  mean texture  mean perimeter  mean area  \
0           0     0.0        17.99         10.38          122.80     1001.0   
1           1     0.0        20.57         17.77          132.90     1326.0   
2           2     0.0        19.69         21.25          130.00     1203.0   
3           3     0.0        11.42         20.38           77.58      386.1   
4           4     0.0        20.29         14.34          135.10     1297.0   
5           5     0.0        12.45         15.70           82.57      477.1   
6           6     0.0        18.25         19.98          119.60     1040.0   
7           7     0.0        13.71         20.83           90.20      577.9   
8           8     0.0        13.00         21.82           87.50      519.8   
9           9     0.0        12.46         24.04           83.97      475.9   

   mean smoothness  mean compactness  mean concavity  mean concave points  \
0          0.11840           0.27760         0.30010 

### 4. Clean-up

#### A. Delete the model

In [23]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

